# Data cleaning
We want the class with labels as numerical value and the body with clean text.

This will remove:
* duplicates
* NaN entires
* non english
* url, html

* make it lowercase
* combine title and body

In [1]:
import pandas as pd
import sys
sys.path.append("../../../scripts_shared/")
from preprocess_text import preprocess_text


In [2]:
# Read CSV into a dataframe
filename = "csv/flutter_testset.csv"
df = pd.read_csv(filename)
df

,id,type,created_at,repo,repo_url,action,title,labels,body
0,1.141754e+10,IssuesEvent,2020-02-03 00:02:35,flutter/flutter,https://api.github.com/repos/flutter/flutter,opened,Keyboard flashes briefly when dismissing multi...,a: quality a: text input f: focus framework,```dart\r\nimport 'package:flutter/material.da...
1,1.141766e+10,IssuesEvent,2020-02-03 00:48:37,flutter/flutter,https://api.github.com/repos/flutter/flutter,opened,"ChocieChip's default ""selected"" style in dark ...",a: accessibility f: material design framework,![flutter_01](https://user-images.githubuserco...
2,1.141771e+10,IssuesEvent,2020-02-03 01:04:00,flutter/flutter,https://api.github.com/repos/flutter/flutter,opened,"Flutter apps restart when you toggle ""Invert C...",dependency: android engine ▣ platform-android,Doesn't look like `android:configChanges` lets...
3,1.141780e+10,IssuesEvent,2020-02-03 01:34:27,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,flutter build output unclear,tool,Some friction using `flutter build`\r\n\r\n1- ...
4,1.141915e+10,IssuesEvent,2020-02-03 07:10:13,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,Ink.image keeps cache of the image somewhere else,p: third party,"I don't know why, but it seems that Ink.image ..."
...,...,...,...,...,...,...,...,...,...
45012,2.060417e+10,IssuesEvent,2022-03-06 18:26:25,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,Yea,team: infra,<!-- Thank you for contributing to Flutter!\n...
45013,2.060462e+10,IssuesEvent,2022-03-06 19:49:39,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,[Impeller] Get rid of //impeller/third_party,engine P4 impeller,The only dependency there is is STB which is o...
45014,2.060486e+10,IssuesEvent,2022-03-06 20:33:15,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,Install log handler for macOS embedder,engine platform-mac a: desktop P4 tech-debt,Rather than relying on the engine's fallback l...
45015,7.334872e+09,IssuesEvent,2018-03-06 00:52:29,flutter/flutter,https://api.github.com/repos/flutter/flutter,opened,a11y of fancy text field,a11y review prod: accessibility,![image](https://user-images.githubusercontent...


In [3]:
# Number or different labels
df.labels.value_counts().to_frame()[:50]

,count
labels,
in triage,6961
waiting for customer response in triage,2248
in triage waiting for customer response,1719
team: infra,1185
waiting for customer response,1041
created via support template,607
created via performance template,469
tool,348
f: material design framework,247


In [4]:
# Split data based on labels. Contains P0 and P1 in one dataframe, the rest in another

pattern = 'P(1|0)'
# Check if 'labels' contains the pattern
hp = df[df['labels'].str.contains(pattern)]
# Reset index
hp = hp.reset_index(drop=True)
hp


/tmp/ipykernel_340011/1635278008.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  hp = df[df['labels'].str.contains(pattern)]


,id,type,created_at,repo,repo_url,action,title,labels,body
0,1.362189e+10,IssuesEvent,2020-09-24 02:01:56,flutter/flutter,https://api.github.com/repos/flutter/flutter,reopened,I can't view build dashboard logs,P1 passed first triage passed secondary triage...,I'm trying to debug https://github.com/flutter...
1,1.362223e+10,IssuesEvent,2020-09-24 03:00:37,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,worst_frame_build_time_millis \tcubic_bezier_p...,P1 f: material design framework passed first t...,https://flutter-flutter-perf.skia.org/e/?begin...
2,1.362854e+10,IssuesEvent,2020-09-24 14:02:05,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,iOS 14 release Invalid Binary related to Inval...,P1 a: release cp: 1.22 e: OS Version specific ...,I've a problem with publishing app to App Stor...
3,1.363092e+10,IssuesEvent,2020-09-24 17:10:38,flutter/flutter,https://api.github.com/repos/flutter/flutter,reopened,iOS 14 release Invalid Binary related to Inval...,P1 a: release cp: 1.22 e: OS Version specific ...,I've a problem with publishing app to App Stor...
4,1.363372e+10,IssuesEvent,2020-09-24 22:01:59,flutter/flutter,https://api.github.com/repos/flutter/flutter,opened,[infra] Mac ios engine builds failing on beta ...,P1 team team: infra,<!-- Thank you for using Flutter!\r\n\r\n ...
...,...,...,...,...,...,...,...,...,...
1710,2.686740e+10,IssuesEvent,2023-02-04 02:57:55,flutter/flutter,https://api.github.com/repos/flutter/flutter,opened,Review request for Revert PR flutter/flutter#1...,P1,Pull request flutter/flutter#119981 was submit...
1711,2.687401e+10,IssuesEvent,2023-02-04 20:29:38,flutter/flutter,https://api.github.com/repos/flutter/flutter,opened,Dart SDK roller blocked on persistent crash,engine dependency: dart P0,Rolls:\r\n* https://github.com/flutter/engine/...
1712,2.687437e+10,IssuesEvent,2023-02-04 21:27:31,flutter/flutter,https://api.github.com/repos/flutter/flutter,opened,High flake rate on web hot reload test `packag...,P0 severe: flake,The high flake rate began around 1PM PST on Fe...
1713,2.687452e+10,IssuesEvent,2023-02-04 21:51:38,flutter/flutter,https://api.github.com/repos/flutter/flutter,opened,High flake rate on `test/integration.shard/de...,P0 severe: flake,Causing many framework tree failures over the ...


In [5]:
# Remove pattern from df
random = df[~df['labels'].str.contains(pattern)]
random = random.reset_index(drop=True)
random

/tmp/ipykernel_340011/2462673340.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  random = df[~df['labels'].str.contains(pattern)]


,id,type,created_at,repo,repo_url,action,title,labels,body
0,1.141754e+10,IssuesEvent,2020-02-03 00:02:35,flutter/flutter,https://api.github.com/repos/flutter/flutter,opened,Keyboard flashes briefly when dismissing multi...,a: quality a: text input f: focus framework,```dart\r\nimport 'package:flutter/material.da...
1,1.141766e+10,IssuesEvent,2020-02-03 00:48:37,flutter/flutter,https://api.github.com/repos/flutter/flutter,opened,"ChocieChip's default ""selected"" style in dark ...",a: accessibility f: material design framework,![flutter_01](https://user-images.githubuserco...
2,1.141771e+10,IssuesEvent,2020-02-03 01:04:00,flutter/flutter,https://api.github.com/repos/flutter/flutter,opened,"Flutter apps restart when you toggle ""Invert C...",dependency: android engine ▣ platform-android,Doesn't look like `android:configChanges` lets...
3,1.141780e+10,IssuesEvent,2020-02-03 01:34:27,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,flutter build output unclear,tool,Some friction using `flutter build`\r\n\r\n1- ...
4,1.141915e+10,IssuesEvent,2020-02-03 07:10:13,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,Ink.image keeps cache of the image somewhere else,p: third party,"I don't know why, but it seems that Ink.image ..."
...,...,...,...,...,...,...,...,...,...
43297,2.060417e+10,IssuesEvent,2022-03-06 18:26:25,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,Yea,team: infra,<!-- Thank you for contributing to Flutter!\n...
43298,2.060462e+10,IssuesEvent,2022-03-06 19:49:39,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,[Impeller] Get rid of //impeller/third_party,engine P4 impeller,The only dependency there is is STB which is o...
43299,2.060486e+10,IssuesEvent,2022-03-06 20:33:15,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,Install log handler for macOS embedder,engine platform-mac a: desktop P4 tech-debt,Rather than relying on the engine's fallback l...
43300,7.334872e+09,IssuesEvent,2018-03-06 00:52:29,flutter/flutter,https://api.github.com/repos/flutter/flutter,opened,a11y of fancy text field,a11y review prod: accessibility,![image](https://user-images.githubusercontent...


In [6]:
# Removing in triage since these issues are not assigned a priority
pattern = 'in triage'
# Remove pattern from df
random = random[~random['labels'].str.contains(pattern)]
random = random.reset_index(drop=True)
random

,id,type,created_at,repo,repo_url,action,title,labels,body
0,1.141754e+10,IssuesEvent,2020-02-03 00:02:35,flutter/flutter,https://api.github.com/repos/flutter/flutter,opened,Keyboard flashes briefly when dismissing multi...,a: quality a: text input f: focus framework,```dart\r\nimport 'package:flutter/material.da...
1,1.141766e+10,IssuesEvent,2020-02-03 00:48:37,flutter/flutter,https://api.github.com/repos/flutter/flutter,opened,"ChocieChip's default ""selected"" style in dark ...",a: accessibility f: material design framework,![flutter_01](https://user-images.githubuserco...
2,1.141771e+10,IssuesEvent,2020-02-03 01:04:00,flutter/flutter,https://api.github.com/repos/flutter/flutter,opened,"Flutter apps restart when you toggle ""Invert C...",dependency: android engine ▣ platform-android,Doesn't look like `android:configChanges` lets...
3,1.141780e+10,IssuesEvent,2020-02-03 01:34:27,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,flutter build output unclear,tool,Some friction using `flutter build`\r\n\r\n1- ...
4,1.141915e+10,IssuesEvent,2020-02-03 07:10:13,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,Ink.image keeps cache of the image somewhere else,p: third party,"I don't know why, but it seems that Ink.image ..."
...,...,...,...,...,...,...,...,...,...
32112,2.060417e+10,IssuesEvent,2022-03-06 18:26:25,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,Yea,team: infra,<!-- Thank you for contributing to Flutter!\n...
32113,2.060462e+10,IssuesEvent,2022-03-06 19:49:39,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,[Impeller] Get rid of //impeller/third_party,engine P4 impeller,The only dependency there is is STB which is o...
32114,2.060486e+10,IssuesEvent,2022-03-06 20:33:15,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,Install log handler for macOS embedder,engine platform-mac a: desktop P4 tech-debt,Rather than relying on the engine's fallback l...
32115,7.334872e+09,IssuesEvent,2018-03-06 00:52:29,flutter/flutter,https://api.github.com/repos/flutter/flutter,opened,a11y of fancy text field,a11y review prod: accessibility,![image](https://user-images.githubusercontent...


In [7]:
random.labels.value_counts().to_frame()[:50]

,count
labels,
team: infra,1185
waiting for customer response,1041
created via support template,607
created via performance template,469
tool,348
f: material design framework,247
engine,213
cp: review,178
team: infra device-lab,159


In [8]:
# Number or different labels
hp.labels.value_counts().to_frame()[:50]

,count
labels,
team: infra P0,125
P1,121
tool team: flakes P1 severe: flake,118
engine team: flakes P1 severe: flake,104
team: flakes P1 severe: flake,63
P0 team: infra,61
P0,57
P1 team: infra,52
engine P0,30


In [9]:
#Give each priority a label by number.
# 'Label encoding'. Makes is easier for machine learning models to work with categorical data.
hp["label"] = 1
hp["class"] = "high_priority"
hp.head()

,id,type,created_at,repo,repo_url,action,title,labels,body,label,class
0,1.362189e+10,IssuesEvent,2020-09-24 02:01:56,flutter/flutter,https://api.github.com/repos/flutter/flutter,reopened,I can't view build dashboard logs,P1 passed first triage passed secondary triage...,I'm trying to debug https://github.com/flutter...,1,high_priority
1,1.362223e+10,IssuesEvent,2020-09-24 03:00:37,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,worst_frame_build_time_millis \tcubic_bezier_p...,P1 f: material design framework passed first t...,https://flutter-flutter-perf.skia.org/e/?begin...,1,high_priority
2,1.362854e+10,IssuesEvent,2020-09-24 14:02:05,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,iOS 14 release Invalid Binary related to Inval...,P1 a: release cp: 1.22 e: OS Version specific ...,I've a problem with publishing app to App Stor...,1,high_priority
3,1.363092e+10,IssuesEvent,2020-09-24 17:10:38,flutter/flutter,https://api.github.com/repos/flutter/flutter,reopened,iOS 14 release Invalid Binary related to Inval...,P1 a: release cp: 1.22 e: OS Version specific ...,I've a problem with publishing app to App Stor...,1,high_priority
4,1.363372e+10,IssuesEvent,2020-09-24 22:01:59,flutter/flutter,https://api.github.com/repos/flutter/flutter,opened,[infra] Mac ios engine builds failing on beta ...,P1 team team: infra,<!-- Thank you for using Flutter!\r\n\r\n ...,1,high_priority


In [10]:
random["label"] = 0
random["class"] = "not_high_priority" 
random.head()

,id,type,created_at,repo,repo_url,action,title,labels,body,label,class
0,1.141754e+10,IssuesEvent,2020-02-03 00:02:35,flutter/flutter,https://api.github.com/repos/flutter/flutter,opened,Keyboard flashes briefly when dismissing multi...,a: quality a: text input f: focus framework,```dart\r\nimport 'package:flutter/material.da...,0,not_high_priority
1,1.141766e+10,IssuesEvent,2020-02-03 00:48:37,flutter/flutter,https://api.github.com/repos/flutter/flutter,opened,"ChocieChip's default ""selected"" style in dark ...",a: accessibility f: material design framework,![flutter_01](https://user-images.githubuserco...,0,not_high_priority
2,1.141771e+10,IssuesEvent,2020-02-03 01:04:00,flutter/flutter,https://api.github.com/repos/flutter/flutter,opened,"Flutter apps restart when you toggle ""Invert C...",dependency: android engine ▣ platform-android,Doesn't look like `android:configChanges` lets...,0,not_high_priority
3,1.141780e+10,IssuesEvent,2020-02-03 01:34:27,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,flutter build output unclear,tool,Some friction using `flutter build`\r\n\r\n1- ...,0,not_high_priority
4,1.141915e+10,IssuesEvent,2020-02-03 07:10:13,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,Ink.image keeps cache of the image somewhere else,p: third party,"I don't know why, but it seems that Ink.image ...",0,not_high_priority


In [11]:
# Drop duplicates by the content of the title
high_priority = hp.drop_duplicates(subset=['title'], keep='last')
high_priority.dropna(inplace=True)
high_priority.reset_index(inplace=True)
high_priority.drop(columns=["index"] , inplace= True)
high_priority["class"].value_counts()

/tmp/ipykernel_340011/1823776403.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_priority.dropna(inplace=True)
/tmp/ipykernel_340011/1823776403.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_priority.drop(columns=["index"] , inplace= True)


class
high_priority    1190
Name: count, dtype: int64

In [12]:
# Drop duplicates by the content of the title
not_high_priority = random.drop_duplicates(subset=['title'], keep='last')
not_high_priority.dropna(inplace=True)
not_high_priority.reset_index(inplace=True)
not_high_priority.drop(columns=["index"] , inplace= True)
not_high_priority["class"].value_counts()

/tmp/ipykernel_340011/3923262596.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_high_priority.dropna(inplace=True)
/tmp/ipykernel_340011/3923262596.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_high_priority.drop(columns=["index"] , inplace= True)


class
not_high_priority    26115
Name: count, dtype: int64

In [13]:
high_priority.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1190 entries, 0 to 1189
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          1190 non-null   float64
 1   type        1190 non-null   object 
 2   created_at  1190 non-null   object 
 3   repo        1190 non-null   object 
 4   repo_url    1190 non-null   object 
 5   action      1190 non-null   object 
 6   title       1190 non-null   object 
 7   labels      1190 non-null   object 
 8   body        1190 non-null   object 
 9   label       1190 non-null   int64  
 10  class       1190 non-null   object 
dtypes: float64(1), int64(1), object(9)
memory usage: 102.4+ KB


In [14]:
label_counts = high_priority["class"].value_counts()
label_counts_nhp = not_high_priority["class"].value_counts()
print(label_counts)
not_high_priority_count = label_counts_nhp["not_high_priority"]
print(not_high_priority_count)
hp_count = label_counts["high_priority"]
hp_count

class
high_priority    1190
Name: count, dtype: int64
26115


1190

In [15]:
not_high_priority = not_high_priority.sample(frac=hp_count/not_high_priority_count, random_state=42)
not_high_priority

,id,type,created_at,repo,repo_url,action,title,labels,body,label,class
9453,2.809583e+10,IssuesEvent,2023-03-30 15:43:44,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,Transition animation curves used by Cupertino ...,framework f: cupertino has reproducible steps ...,CupertinoPageRoute animations are now outdated...,0,not_high_priority
7826,9.995683e+09,IssuesEvent,2019-07-11 20:54:53,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,Table widget doesn't work with Hero animations.,customer: payouts framework severe: crash,## Steps to reproduce\r\n\r\nRun the following...,0,not_high_priority
14649,1.020933e+10,IssuesEvent,2019-08-14 12:27:55,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,Content Overflows when using NestedScrollView ...,f: material design f: scrolling framework,"In the below code, the `_menuPage()` returns a...",0,not_high_priority
10947,1.282326e+10,IssuesEvent,2020-07-06 11:21:56,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,Error connecting to the service protocol: fail...,P4 platform-ios severe: crash t: xcode tool,App launch fails: physical device-6th Gen iPad...,0,not_high_priority
20220,1.055266e+10,IssuesEvent,2019-10-03 15:35:01,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,Firestore Calculations not working,p: cloud_firestore plugin,### Objective\r\nI've got a Firestore app that...,0,not_high_priority
...,...,...,...,...,...,...,...,...,...,...,...
10863,7.174676e+09,IssuesEvent,2018-01-31 00:51:45,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,"Gallery, Scrollable tabs: Open and Close miss ...",customer blocker performance prod: framework t...,Many of the component demos open and close qui...,0,not_high_priority
25824,1.031519e+10,IssuesEvent,2019-08-30 06:47:12,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,TextEditingController.clear() makes keyboard u...,a: text input framework severe: regression,<!-- Thank you for using Flutter!\r\n\r\n ...,0,not_high_priority
23239,2.646927e+10,IssuesEvent,2023-01-17 05:07:02,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,DeviceOrientation.landscapeRight is upside dow...,platform-android engine a: layout has reproduc...,<!-- Thank you for using Flutter!\r\n\r\n ...,0,not_high_priority
280,2.523019e+10,IssuesEvent,2022-11-14 19:07:52,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,Exception: Failed to access the internal image...,severe: crash engine a: images has reproducibl...,I encountered the following error trying use t...,0,not_high_priority


In [16]:
all_priority = pd.concat([high_priority,not_high_priority] , ignore_index = True)
all_priority.tail()

,id,type,created_at,repo,repo_url,action,title,labels,body,label,class
2375,7.174676e+09,IssuesEvent,2018-01-31 00:51:45,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,"Gallery, Scrollable tabs: Open and Close miss ...",customer blocker performance prod: framework t...,Many of the component demos open and close qui...,0,not_high_priority
2376,1.031519e+10,IssuesEvent,2019-08-30 06:47:12,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,TextEditingController.clear() makes keyboard u...,a: text input framework severe: regression,<!-- Thank you for using Flutter!\r\n\r\n ...,0,not_high_priority
2377,2.646927e+10,IssuesEvent,2023-01-17 05:07:02,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,DeviceOrientation.landscapeRight is upside dow...,platform-android engine a: layout has reproduc...,<!-- Thank you for using Flutter!\r\n\r\n ...,0,not_high_priority
2378,2.523019e+10,IssuesEvent,2022-11-14 19:07:52,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,Exception: Failed to access the internal image...,severe: crash engine a: images has reproducibl...,I encountered the following error trying use t...,0,not_high_priority
2379,2.412155e+10,IssuesEvent,2022-09-20 19:11:56,flutter/flutter,https://api.github.com/repos/flutter/flutter,reopened,[pigeon] Can't be tested on stable channel as ...,platform-android platform-ios passed first tri...,Due to unrecognized version to the Dart SDK on...,0,not_high_priority


In [17]:
# 
all_priority["label"].value_counts()

label
1    1190
0    1190
Name: count, dtype: int64

In [18]:
print(all_priority["title"][0])
print(all_priority["body"][0])

worst_frame_build_time_millis 	cubic_bezier_perf__e2e_summary regression
https://flutter-flutter-perf.skia.org/e/?begin=1600207024&end=1600291175&keys=X197f56aeb2671ed26940c2a2de56e74d&num_commits=50&request_type=1&xbaroffset=21489.

Looks like this commit: https://github.com/flutter/flutter/pull/65900. I suspect given the commit that this is expected?

/cc @HansMuller 




In [19]:
# Copy content of body to a new col named text
all_priority["text"] = all_priority["title"] + all_priority["body"]
all_priority.tail()

,id,type,created_at,repo,repo_url,action,title,labels,body,label,class,text
2375,7.174676e+09,IssuesEvent,2018-01-31 00:51:45,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,"Gallery, Scrollable tabs: Open and Close miss ...",customer blocker performance prod: framework t...,Many of the component demos open and close qui...,0,not_high_priority,"Gallery, Scrollable tabs: Open and Close miss ..."
2376,1.031519e+10,IssuesEvent,2019-08-30 06:47:12,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,TextEditingController.clear() makes keyboard u...,a: text input framework severe: regression,<!-- Thank you for using Flutter!\r\n\r\n ...,0,not_high_priority,TextEditingController.clear() makes keyboard u...
2377,2.646927e+10,IssuesEvent,2023-01-17 05:07:02,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,DeviceOrientation.landscapeRight is upside dow...,platform-android engine a: layout has reproduc...,<!-- Thank you for using Flutter!\r\n\r\n ...,0,not_high_priority,DeviceOrientation.landscapeRight is upside dow...
2378,2.523019e+10,IssuesEvent,2022-11-14 19:07:52,flutter/flutter,https://api.github.com/repos/flutter/flutter,closed,Exception: Failed to access the internal image...,severe: crash engine a: images has reproducibl...,I encountered the following error trying use t...,0,not_high_priority,Exception: Failed to access the internal image...
2379,2.412155e+10,IssuesEvent,2022-09-20 19:11:56,flutter/flutter,https://api.github.com/repos/flutter/flutter,reopened,[pigeon] Can't be tested on stable channel as ...,platform-android platform-ios passed first tri...,Due to unrecognized version to the Dart SDK on...,0,not_high_priority,[pigeon] Can't be tested on stable channel as ...


In [20]:
all_priority["text"][0]

'worst_frame_build_time_millis \tcubic_bezier_perf__e2e_summary regressionhttps://flutter-flutter-perf.skia.org/e/?begin=1600207024&end=1600291175&keys=X197f56aeb2671ed26940c2a2de56e74d&num_commits=50&request_type=1&xbaroffset=21489.\r\n\r\nLooks like this commit: https://github.com/flutter/flutter/pull/65900. I suspect given the commit that this is expected?\r\n\r\n/cc @HansMuller \r\n\r\n'

In [21]:
# Make a new dataframe with only text, label and class cols.
all_priority_subset = all_priority[["text" , "label" , "class"]]
all_priority_subset

,text,label,class
0,worst_frame_build_time_millis \tcubic_bezier_p...,1,high_priority
1,iOS 14 release Invalid Binary related to Inval...,1,high_priority
2,[infra] Mac ios engine builds failing on beta ...,1,high_priority
3,Review request for Revert PR flutter/flutter#1...,1,high_priority
4,Flutter to Plugin Roll broken by removal of `F...,1,high_priority
...,...,...,...
2375,"Gallery, Scrollable tabs: Open and Close miss ...",0,not_high_priority
2376,TextEditingController.clear() makes keyboard u...,0,not_high_priority
2377,DeviceOrientation.landscapeRight is upside dow...,0,not_high_priority
2378,Exception: Failed to access the internal image...,0,not_high_priority


In [22]:
# Convert to string
all_priority_subset["text_str"] = all_priority_subset['text'].astype(str)

/tmp/ipykernel_340011/3106036705.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_priority_subset["text_str"] = all_priority_subset['text'].astype(str)


In [23]:
all_priority_subset

,text,label,class,text_str
0,worst_frame_build_time_millis \tcubic_bezier_p...,1,high_priority,worst_frame_build_time_millis \tcubic_bezier_p...
1,iOS 14 release Invalid Binary related to Inval...,1,high_priority,iOS 14 release Invalid Binary related to Inval...
2,[infra] Mac ios engine builds failing on beta ...,1,high_priority,[infra] Mac ios engine builds failing on beta ...
3,Review request for Revert PR flutter/flutter#1...,1,high_priority,Review request for Revert PR flutter/flutter#1...
4,Flutter to Plugin Roll broken by removal of `F...,1,high_priority,Flutter to Plugin Roll broken by removal of `F...
...,...,...,...,...
2375,"Gallery, Scrollable tabs: Open and Close miss ...",0,not_high_priority,"Gallery, Scrollable tabs: Open and Close miss ..."
2376,TextEditingController.clear() makes keyboard u...,0,not_high_priority,TextEditingController.clear() makes keyboard u...
2377,DeviceOrientation.landscapeRight is upside dow...,0,not_high_priority,DeviceOrientation.landscapeRight is upside dow...
2378,Exception: Failed to access the internal image...,0,not_high_priority,Exception: Failed to access the internal image...


In [24]:
# Clean the data.
all_priority_subset["text_clean"] = all_priority_subset["text_str"].map(preprocess_text)

/tmp/ipykernel_340011/63294665.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_priority_subset["text_clean"] = all_priority_subset["text_str"].map(preprocess_text)


In [25]:
# Make a subset with text_clean and label
priority_label_text = all_priority_subset[["text_clean" , "label"]]
priority_label_text

,text_clean,label
0,worstframebuildtimemillis regression looks lik...,1
1,ios release invalid binary related to invalid ...,1
2,mac ios engine builds failing on beta branch t...,1
3,review request for revert pr request was submi...,1
4,flutter to plugin roll broken by removal of fl...,1
...,...,...
2375,gallery scrollable tabs open and close miss fr...,0
2376,texteditingcontrollerclear makes keyboard unre...,0
2377,deviceorientationlandscaperight is upside down...,0
2378,exception failed to access the internal image ...,0


In [26]:
# Need to dropna here since cleaning function returns NaN for not english text.
priority_label_text.dropna(inplace=True)
priority_label_text.reset_index(inplace=True)
priority_label_text.drop(columns=["index"] , inplace= True)

priority_label_text

/tmp/ipykernel_340011/239147930.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  priority_label_text.dropna(inplace=True)
/tmp/ipykernel_340011/239147930.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  priority_label_text.drop(columns=["index"] , inplace= True)


,text_clean,label
0,worstframebuildtimemillis regression looks lik...,1
1,ios release invalid binary related to invalid ...,1
2,mac ios engine builds failing on beta branch t...,1
3,review request for revert pr request was submi...,1
4,flutter to plugin roll broken by removal of fl...,1
...,...,...
2368,gallery scrollable tabs open and close miss fr...,0
2369,texteditingcontrollerclear makes keyboard unre...,0
2370,deviceorientationlandscaperight is upside down...,0
2371,exception failed to access the internal image ...,0


In [27]:
from sklearn.utils import resample

# Split the dataset into two based on the label
df_majority = priority_label_text[priority_label_text['label'] == 1]
df_minority = priority_label_text[priority_label_text['label'] == 0]

# Undersample the majority class
df_majority_undersampled = resample(df_majority, 
                                     replace=False,    # sample without replacement
                                     n_samples=len(df_minority),     # to match minority class
                                     random_state=123) # reproducible results

# Combine minority class with downsampled majority class
df_balanced = pd.concat([df_majority_undersampled, df_minority])

# Shuffle the dataset to avoid any ordering bias
df_balanced = df_balanced.sample(frac=1).reset_index(drop=True)
priority_label_text = df_balanced
priority_label_text["label"].value_counts()

label
0    1186
1    1186
Name: count, dtype: int64

In [28]:
# Clean dataset with clean text and labels.
# 0 = high priority, 1 = not high priority
file_name = f"csv/clean_flutter_testset.csv"
priority_label_text.to_csv(file_name, index=False)

In [29]:
pri = pd.read_csv(file_name)
pri

,text_clean,label
0,some errors are silencedi need some help in fi...,0
1,flutter not rendering text after sitting idle ...,1
2,windows frameworktestslibraries is flaky metat...,1
3,reverse listview scrolls to bottom when tappin...,0
4,iconbutton doc link point to the wrong website...,0
...,...,...
2367,we should crash if you both enableimpeller and...,1
2368,tree closed on infra failuresfor example,1
2369,temporarily disable windows tests try and prod...,1
2370,linux routingtest infra failurei cant tell wha...,1
